In [1]:
# Import libraries
import sqlalchemy
import pandas as pd
from  psycopg2 import connect
import os
from config import *
from sqlalchemy import Column, ForeignKey,Table, Integer, String, Float, Boolean, MetaData, create_engine

In [2]:
# Input file paths
cleaned_tweet_file=os.path.join("","..","Resources","outputData","tweetCleandata.csv")
predict_tweet_file=os.path.join("","..","Resources","outputData","tweetPredData.csv")

In [3]:
# Load tweet csv file
tweet_clean_df= pd.read_csv(cleaned_tweet_file)
tweet_clean_df.head()

,TweetID,Date,Matched Keywords,User,Source,Followers,Friends,Favorite,Tweet,predTweet,CleanedTweet
0,1292795662485131264,08/10/2020,Trump,MenshevikM,Twitter Web App,5882.0,320.0,0.0,They're even right that there's a certain kind...,they are even right that there certain kind of...,"['right', 'certain', 'liber', 'deeply', 'want'..."
1,1292795661809852417,08/10/2020,Trump,balling_it,Twitter Web App,33.0,156.0,0.0,"@jonathanchait Saw, that is the press, people ...",naw that the press people who encouraged voter...,"['press', 'people', 'encourage', 'voter', 'tru..."
2,1292795659704242181,08/10/2020,Trump,laurie71,Twitter for iPhone,85.0,141.0,0.0,@sarahcpr Trump signs an executive order to th...,trump signs an executive order throw rotted sc...,"['trump', 'sign', 'execute', 'order', 'throw',..."
3,1292795658747944960,08/10/2020,Biden,bluewave4peace,Twitter for iPhone,528.0,745.0,0.0,@glennkirschner2 Sorry I want to be on a real ...,sorry want be real team truthful team justice ...,"['sorry', 'truth', 'justice', 'equal', 'win', ..."
4,1292795658550812672,08/10/2020,Trump,OGOPer,Twitter for iPhone,2839.0,2655.0,0.0,Year. Until Ben Masse cowers and yelps before ...,yeah until ben sasse cowers and yelps before j...,"['same', 'cower', 'yelp', 'before', 'jump', 'e..."


In [4]:
# Load tweet csv file
tweet_predict_df= pd.read_csv(predict_tweet_file)
tweet_predict_df.head()

,TweetID,Date,Matched Keywords,User,Source,Followers,Friends,Favorite,Tweet,predTweet,CleanedTweet,Prediction
0,1292795662485131264,08/10/2020,Trump,MenshevikM,Twitter Web App,5882.0,320.0,0.0,They're even right that there's a certain kind...,they are even right that there certain kind of...,"['right', 'certain', 'liber', 'deeply', 'want'...",Negative
1,1292795661809852417,08/10/2020,Trump,balling_it,Twitter Web App,33.0,156.0,0.0,"@jonathanchait Saw, that is the press, people ...",naw that the press people who encouraged voter...,"['press', 'people', 'encourage', 'voter', 'tru...",Positive
2,1292795659704242181,08/10/2020,Trump,laurie71,Twitter for iPhone,85.0,141.0,0.0,@sarahcpr Trump signs an executive order to th...,trump signs an executive order throw rotted sc...,"['trump', 'sign', 'execute', 'order', 'throw',...",Negative
3,1292795658747944960,08/10/2020,Biden,bluewave4peace,Twitter for iPhone,528.0,745.0,0.0,@glennkirschner2 Sorry I want to be on a real ...,sorry want be real team truthful team justice ...,"['sorry', 'truth', 'justice', 'equal', 'win', ...",Positive
4,1292795658550812672,08/10/2020,Trump,OGOPer,Twitter for iPhone,2839.0,2655.0,0.0,Year. Until Ben Masse cowers and yelps before ...,yeah until ben sasse cowers and yelps before j...,"['same', 'cower', 'yelp', 'before', 'jump', 'e...",Negative


In [5]:
# Create table specific data frames

In [6]:
tweet_user_df= tweet_clean_df[['TweetID','Date','Matched Keywords','User','Followers','Friends','Favorite']]

In [7]:
tweet_source_df = tweet_clean_df[['TweetID','Date','Matched Keywords','Source']]

In [8]:
tweet_df = tweet_predict_df[['TweetID','Date','Matched Keywords','Tweet','Prediction']]

In [9]:
tweet_clean_df = tweet_clean_df[['TweetID','Date','Matched Keywords','CleanedTweet']]

In [10]:
tweet_predict_df = tweet_predict_df[['TweetID','Date','Matched Keywords','predTweet','Prediction']]

In [11]:
# Database connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbName}')

In [12]:
# Create MetaData
meta = MetaData(engine)

In [13]:
# 1. Twitter User Table
twitter_user = Table('tweetUser', meta,
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('User', String),
    Column('Followers', Integer),
    Column('Friends', Integer),
    Column('Favorite', Integer))

meta.create_all(engine)

tweet_user_df.to_sql(name='tweetUser', con=engine, if_exists='append', index=False)

In [14]:
# 2. Twitte Source Table
twitter_source = Table('twitterSource', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('Source', String))

meta.create_all(engine)

tweet_source_df.to_sql(name='twitterSource', con=engine, if_exists='append', index=False)

In [15]:
# 3. Twitte Data Table
twitter_data = Table('tweetsData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('Tweet', String),
    Column('Prediction', String))

meta.create_all(engine)

tweet_df.to_sql(name='tweetsData', con=engine, if_exists='append', index=False)

In [16]:
# 4. Twitter Clean Data Table
twitter_clean_data = Table('tweetsCleanData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('CleanedTweet', String))

meta.create_all(engine)

tweet_clean_df.to_sql(name='tweetsCleanData', con=engine, if_exists='append', index=False)

In [17]:
# 5. Twitter Prediction Data Table
twitter_predict_data = Table('tweetsPredictData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('predTweet', String),
    Column('Prediction', String))

meta.create_all(engine)

tweet_predict_df.to_sql(name='tweetsPredictData', con=engine, if_exists='append', index=False)

In [18]:
engine.table_names()

['tweetUser',
 'twitterSource',
 'tweetsData',
 'tweetsCleanData',
 'tweetsPredictData']